We'll start with basics before anything else fancy... including in basics are doing poly and log transformations, however I'm not planning on segmenitng the data again... unless the basic linear just does that good then I don't want to, considerng how poor it did last time.

In [61]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler #Temp

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV, RFE #get rid of rfecv later?

In [4]:
df = pd.read_csv('../data/train.csv')

df.columns = df.columns.str.lower() #Technically a change, but I won't bother to re-code it.
df = df.drop('id', axis=1)
#I honestly forgot to earlier, although I do like the data is preserved: At least at the moment we also need to dummify sex
#to get our lovely sklearn methods to work:

print(df.shape)
df.head()

(1117957, 21)


,monsoonintensity,topographydrainage,rivermanagement,deforestation,urbanization,climatechange,damsquality,siltation,agriculturalpractices,encroachments,...,drainagesystems,coastalvulnerability,landslides,watersheds,deterioratinginfrastructure,populationscore,wetlandloss,inadequateplanning,politicalfactors,floodprobability
0,5,8,5,8,6,4,4,3,3,4,...,5,3,3,5,4,7,5,7,3,0.445
1,6,7,4,4,8,8,3,5,4,6,...,7,2,0,3,5,3,3,4,3,0.450
2,6,5,6,7,3,7,1,5,4,5,...,7,3,7,5,6,8,2,3,3,0.530
3,3,4,6,5,4,8,4,7,6,8,...,2,4,7,4,4,6,5,7,5,0.535
4,5,3,2,6,4,4,3,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415


In [5]:
X = df.drop('floodprobability', axis=1)
y = df['floodprobability']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=26)
#Once again we'll have a much larger test size, give how large the data is...
#Re. 26 - I like that number. Discussion not for now

ss = StandardScaler() #Considering the scale likely negligible... but eh, habit. Plus, it likely will be relevant when we
#polify (sp?) everything momentarily.
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

I think I jumped the gun last time, rushing to rfe already. We'll work with all of them for now and play around with
what to remove later. Ie, I'll assume a certain consistency about models (and by default going forward):

Trends should be consistent throughout models. Ie, even if forests are the way to go let's assume the difference between
regular regression and forests are akin to logged regression and logged forests (heh). Ie, we'll start with the 'basic manipulations' then go to the various algorithims later.

In [10]:
lr = LinearRegression()

current_model = lr.fit(X_train, y_train)

current_preds = lr.predict(X_test)

print(current_model.score(X_train, y_train), current_model.score(X_test, y_test))
print(cross_val_score(lr, X_train, y_train, cv = 5).mean())#Hmm, nice and if anything slightly underfit.

0.8449249636091961 0.8450060356811127
0.8449155523743663


In [49]:
#Now with 20 features bouncing off each other likely we'll see some overfitting. But, I kinda want that...

X = df.drop('floodprobability', axis=1)
y = df['floodprobability']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=26)

poly = PolynomialFeatures(degree = 2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

df_poly_train = pd.DataFrame(X_train_poly, columns=poly.get_feature_names_out(X.columns)) #We'll use this later
df_poly_test = pd.DataFrame(X_test_poly, columns=poly.get_feature_names_out(X.columns))

ss = StandardScaler()
X_train_poly = ss.fit_transform(X_train_poly)
X_test_poly = ss.transform(X_test_poly)

print(len(poly.get_feature_names_out(X.columns))) #Yeah, sounds about right... we'll still shove everything in there
#for an initial run though.

230


In [17]:
#Hmm, might be a move later to subsitute X_{Type} for a variable at the begining.

lr = LinearRegression()

current_model = lr.fit(X_train_poly, y_train)

current_preds = lr.predict(X_test_poly)

print(current_model.score(X_train_poly, y_train), current_model.score(X_test_poly, y_test))
print(cross_val_score(lr, X_train_poly, y_train, cv = 5).mean()) #Slightly higher and quite shockiingly not overfit,
#however, I'd venture to assume that surely something is superfluous... but for now we continue.

0.8455553546245052 0.8455353777086142
0.8454452816829896


In [60]:
#Fine, so given our current log_roadblock let's play around with these features.

#Note, at least 20 takes waaaay too long to run on my computer, but still giving us a .845...worth a try though.

# lr = LinearRegression()
# rfe = RFE(lr, step=1, n_features_to_select=20)

# current_model = rfe.fit(X_train_poly, y_train)

# current_preds = rfe.predict(X_test_poly)

# print(current_model.score(X_train_poly, y_train), current_model.score(X_test_poly, y_test))
# print(cross_val_score(rfe, X_train_poly, y_train, cv = 5).mean())
#Took quite some time to run so I wouldn't even bother again... Hmm, guess maybe we'll be lazy and just use ridge.

#Plus, I really don't understand what's up with the differences betwen the quite poor, but at least similar, overall scors
#versus the crossval; oh gf me - the same one as before... ie do it with rfe...

In [63]:
#Note that I'm quite unsure if ridge will even matter, given that it's not overfit. But, we might as well try.

# r_alphas = np.logspace(0, 5,100)
# ridge = RidgeCV(alphas=r_alphas, cv = 5, scoring='r2')

# current_model = ridge.fit(X_train_poly, y_train)

# current_preds = ridge.predict(X_test_poly)

# print(current_model.score(X_train_poly, y_train), current_model.score(X_test_poly, y_test))
# print(cross_val_score(ridge, 
#                       X_train_poly, y_train, cv = 5).mean())

#Meh, also .85ish...plus this ran way too long

## Logs

In [47]:
##Unfortunately I still haven't gotten logs to work yet... will try to approach them later.

# df_log = np.log(df)
# df_log.replace('-inf', 5, inplace=True) #Recall that 0s before -infs... so to pragmatically work with them...

# X = df_log.drop('floodprobability', axis=1)
# y = df_log['floodprobability']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=26)

# ss = StandardScaler() 
# X_train = ss.fit_transform(X_train)
# X_test = ss.transform(X_test)

#A bit confused about the following error: ValueError: Input X contains infinity or a value too large for dtype('float64').
#So I'll just assume that any skewness, which we really didn't have, it negligible...see above.

In [48]:
# lr = LinearRegression()

# current_model = lr.fit(X_train, y_train)

# current_preds = lr.predict(X_test)

# print(current_model.score(X_train, y_train), current_model.score(X_test, y_test))
# print(cross_val_score(lr, X_train, y_train, cv = 5).mean())